In [122]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
import array
import math
import statsmodels.stats.multitest as multi

In [123]:
healthy_df = pd.read_csv("lusc-rsem-fpkm-tcga_paired.txt", sep = '\t')

In [124]:
healthy_df.head()

,Hugo_Symbol,Entrez_Gene_Id,TCGA-43-7657,TCGA-58-8386,TCGA-22-5478,TCGA-22-5472,TCGA-43-5670,TCGA-60-2709,TCGA-22-5489,TCGA-77-8007,...,TCGA-85-7710,TCGA-56-7580,TCGA-43-6647,TCGA-90-6837,TCGA-56-8083,TCGA-51-4079,TCGA-56-7222,TCGA-22-4593,TCGA-51-4081,TCGA-56-8309
0,HIST3H2A,92815,62.12,130.60,33.06,35.50,73.03,60.39,92.05,66.65,...,90.77,59.55,40.07,22.92,29.91,82.29,4.70,37.32,43.63,77.25
1,LIN7B,64130,185.11,283.05,119.26,169.07,165.57,161.02,131.51,198.47,...,185.11,119.26,102.97,123.50,264.03,194.36,166.73,105.15,185.11,356.05
2,LXN,56925,909.17,819.30,412.00,743.43,1340.84,607.87,1709.26,1709.26,...,813.63,2400.97,543.96,2193.99,540.19,521.76,253.23,764.36,518.15,878.17
3,CNKSR2,22866,41.81,18.29,40.93,67.12,54.72,29.27,20.26,23.76,...,34.51,70.01,57.49,57.89,67.12,34.51,22.10,31.90,28.24,49.91
4,SCML1,6322,133.36,214.27,108.14,109.66,190.34,211.31,96.01,208.38,...,251.48,209.84,120.10,109.66,155.50,162.14,277.20,86.43,164.42,155.50


In [125]:
cancer_df = pd.read_csv("lusc-rsem-fpkm-tcga-t_paired.txt", sep = '\t')

In [126]:
cancer_df.head()

,Hugo_Symbol,Entrez_Gene_Id,TCGA-43-7657,TCGA-58-8386,TCGA-22-5478,TCGA-22-5472,TCGA-43-5670,TCGA-60-2709,TCGA-22-5489,TCGA-77-8007,...,TCGA-85-7710,TCGA-56-7580,TCGA-43-6647,TCGA-90-6837,TCGA-56-8083,TCGA-51-4079,TCGA-56-7222,TCGA-22-4593,TCGA-51-4081,TCGA-56-8309
0,HIST3H2A,92815,336.79,500.46,703.28,287.01,486.75,70.51,145.02,14.03,...,3.06,420.68,109.66,106.63,1233.75,172.65,303.44,228.13,251.48,23.59
1,LIN7B,64130,105.15,212.78,102.25,212.78,172.65,244.57,105.89,152.28,...,135.24,135.24,151.22,395.18,295.11,120.94,114.36,96.68,277.20,153.34
2,LXN,56925,848.22,236.21,271.48,759.08,61.25,620.67,329.84,599.49,...,688.78,204.07,438.59,503.95,3039.30,607.87,106.63,507.46,255.00,346.29
3,CNKSR2,22866,32.59,8.51,45.85,6.16,49.21,11.91,12.27,15.00,...,1.38,6.62,6.11,1.66,33.54,3.11,0.82,3.32,27.25,6.16
4,SCML1,6322,84.63,74.58,67.12,57.89,102.97,132.44,66.65,57.08,...,165.57,119.26,87.65,53.57,232.94,67.12,64.80,90.14,198.47,154.42


In [127]:
ZERO_THRESHOLD = 30

In [128]:
def contain_zeros(h_series, c_series):
    """
    contain_zeros: a function that counts the zeros in both cancer and healthy series and
    returns True if either exceeds ZERO_THRESHOLD, False otherwise
    """
    h_zeros = 0
    c_zeros = 0
    h_value_counts = h_series.value_counts()
    c_value_counts = c_series.value_counts()
    if 0 in h_value_counts.index:
        h_zeros = h_value_counts[0]
    else:
        h_zeros = 0
    if 0 in c_value_counts.index:
        c_zeros = c_value_counts[0]
    else:
        c_zeros = 0
        
    return (h_zeros > ZERO_THRESHOLD or c_zeros > ZERO_THRESHOLD)

In [129]:
p_value_dict = {}
for i in range(healthy_df.shape[0]):
    gene_type = healthy_df.iloc[i, 0]
    healthy_values = healthy_df.iloc[i, 2:]
    cancer_values = cancer_df.iloc[i, 2:]
    zeros_status = contain_zeros(healthy_values, cancer_values)
    if not zeros_status: # if the number of zeros in both series doesn't exceed 30
        p_value= ttest_ind(healthy_values, cancer_values).pvalue
        if not math.isnan(p_value): # to ensure that correlation is not null-valued
            p_value_dict[gene_type] = p_value

In [130]:
p_value_dict

{'HIST3H2A': 3.6071401066563956e-09,
 'LIN7B': 0.313829482510041,
 'LXN': 8.164043644485789e-05,
 'CNKSR2': 6.3746517799747675e-15,
 'SCML1': 0.04726589843891226,
 'AC024592.12': 0.060316150492081104,
 'GSDMD': 5.3442886577203215e-06,
 'AKR1C1': 7.857877107856742e-06,
 'C3orf62': 1.5007152753534474e-09,
 'CRISPLD2': 9.742753347580845e-05,
 'DOCK5': 0.04671578879072979,
 'SLC33A1': 6.382057732213531e-11,
 'GLI1': 0.02398039841739113,
 'STK17B': 5.024134511809233e-13,
 'RP5-966M1.6': 1.7233551280507414e-09,
 'VPS52': 0.6096108831192086,
 'HAGHL': 0.009115074132559409,
 'CDC37L1': 3.156672288692492e-16,
 'KRT10': 0.0003716169034954104,
 'AEN': 0.6618238617557821,
 'CYHR1': 0.0002779600603376105,
 'VWA1': 0.35764236729681353,
 'EXOSC9': 0.2146744760479479,
 'TNFAIP8L2': 5.504189276736617e-23,
 'PFDN2': 2.225416254484484e-16,
 'TCP1': 9.875245155160638e-12,
 'ALG9': 3.260870772945752e-08,
 'RPS15A': 0.6025969151731205,
 'PMEL': 0.013851285748684753,
 'SFMBT1': 0.2428096710632932,
 'CYBA': 9

In [132]:
rejected_genes={}
for gene, pvalue in p_value_dict.items():
    if(pvalue <= 0.05):
        rejected_genes[gene]='Rejeced H0'
#     else:
#         print(gene, "failed to reject Ho")

In [133]:
rejected_genes

{'HIST3H2A': 'Rejeced H0',
 'LXN': 'Rejeced H0',
 'CNKSR2': 'Rejeced H0',
 'SCML1': 'Rejeced H0',
 'GSDMD': 'Rejeced H0',
 'AKR1C1': 'Rejeced H0',
 'C3orf62': 'Rejeced H0',
 'CRISPLD2': 'Rejeced H0',
 'DOCK5': 'Rejeced H0',
 'SLC33A1': 'Rejeced H0',
 'GLI1': 'Rejeced H0',
 'STK17B': 'Rejeced H0',
 'RP5-966M1.6': 'Rejeced H0',
 'HAGHL': 'Rejeced H0',
 'CDC37L1': 'Rejeced H0',
 'KRT10': 'Rejeced H0',
 'CYHR1': 'Rejeced H0',
 'TNFAIP8L2': 'Rejeced H0',
 'PFDN2': 'Rejeced H0',
 'TCP1': 'Rejeced H0',
 'ALG9': 'Rejeced H0',
 'PMEL': 'Rejeced H0',
 'CYBA': 'Rejeced H0',
 'GINS1': 'Rejeced H0',
 'GPX3': 'Rejeced H0',
 'IPO9': 'Rejeced H0',
 'SHKBP1': 'Rejeced H0',
 'ARHGAP40': 'Rejeced H0',
 'ENO3': 'Rejeced H0',
 'COX19': 'Rejeced H0',
 'LMF1': 'Rejeced H0',
 'GDE1': 'Rejeced H0',
 'LSM14B': 'Rejeced H0',
 'TLE2': 'Rejeced H0',
 'EIF4EBP3': 'Rejeced H0',
 'LRP2BP': 'Rejeced H0',
 'PELO': 'Rejeced H0',
 'CDIPT': 'Rejeced H0',
 'C19orf44': 'Rejeced H0',
 'PABPN1': 'Rejeced H0',
 'ZWINT': 'Rejec

In [134]:
len(rejected_genes)

12799

In [153]:
p_value_rel_dict = {}
for i in range(healthy_df.shape[0]):
    gene_type = healthy_df.iloc[i, 0]
    healthy_values = healthy_df.iloc[i, 2:]
    cancer_values = cancer_df.iloc[i, 2:]
    zeros_status = contain_zeros(healthy_values, cancer_values)
    if not zeros_status: # if the number of zeros in both series doesn't exceed 30
        p_value_rel= ttest_rel(healthy_values, cancer_values).pvalue
        if not math.isnan(p_value): # to ensure that correlation is not null-valued
            p_value_rel_dict[gene_type] = p_value_rel

In [154]:
p_value_rel_dict

{'HIST3H2A': 4.0436066512253253e-08,
 'LIN7B': 0.2891645967596114,
 'LXN': 0.0002322367302316609,
 'CNKSR2': 3.420577429814986e-12,
 'SCML1': 0.06251345969931547,
 'AC024592.12': 0.08658823731308368,
 'GSDMD': 3.041721155197623e-06,
 'AKR1C1': 1.9385746866608284e-05,
 'C3orf62': 4.768559345720827e-11,
 'CRISPLD2': 1.3766122667116988e-05,
 'DOCK5': 0.06415312266683633,
 'SLC33A1': 7.584453673521912e-10,
 'GLI1': 0.02130179835131911,
 'STK17B': 4.51183332012357e-13,
 'RP5-966M1.6': 1.6678484321488395e-09,
 'VPS52': 0.6148682201927615,
 'HAGHL': 0.004337619102628813,
 'CDC37L1': 6.001366612356978e-12,
 'KRT10': 0.0005784370532957748,
 'AEN': 0.652979252379783,
 'CYHR1': 0.00013726998677642208,
 'VWA1': 0.3726066525613332,
 'EXOSC9': 0.18811884631854897,
 'TNFAIP8L2': 1.795963782588147e-18,
 'PFDN2': 2.7764211853232284e-13,
 'TCP1': 6.667076623632467e-10,
 'ALG9': 9.061554520749966e-08,
 'RPS15A': 0.6204247342604798,
 'PMEL': 0.01460936770378406,
 'SFMBT1': 0.26141464389270735,
 'CYBA': 8.

In [155]:
len(p_value_rel_dict)

17653

In [156]:
rejected_genes_paired={}
for gene,p_value in p_value_rel_dict.items():
    if(pvalue <= 0.05):
        rejected_genes_paired[gene]='Rejeced H0'
#     else:
#         print(gene, "failed to reject Ho")

In [157]:
rejected_genes_paired

{'HIST3H2A': 'Rejeced H0',
 'LIN7B': 'Rejeced H0',
 'LXN': 'Rejeced H0',
 'CNKSR2': 'Rejeced H0',
 'SCML1': 'Rejeced H0',
 'AC024592.12': 'Rejeced H0',
 'GSDMD': 'Rejeced H0',
 'AKR1C1': 'Rejeced H0',
 'C3orf62': 'Rejeced H0',
 'CRISPLD2': 'Rejeced H0',
 'DOCK5': 'Rejeced H0',
 'SLC33A1': 'Rejeced H0',
 'GLI1': 'Rejeced H0',
 'STK17B': 'Rejeced H0',
 'RP5-966M1.6': 'Rejeced H0',
 'VPS52': 'Rejeced H0',
 'HAGHL': 'Rejeced H0',
 'CDC37L1': 'Rejeced H0',
 'KRT10': 'Rejeced H0',
 'AEN': 'Rejeced H0',
 'CYHR1': 'Rejeced H0',
 'VWA1': 'Rejeced H0',
 'EXOSC9': 'Rejeced H0',
 'TNFAIP8L2': 'Rejeced H0',
 'PFDN2': 'Rejeced H0',
 'TCP1': 'Rejeced H0',
 'ALG9': 'Rejeced H0',
 'RPS15A': 'Rejeced H0',
 'PMEL': 'Rejeced H0',
 'SFMBT1': 'Rejeced H0',
 'CYBA': 'Rejeced H0',
 'GINS1': 'Rejeced H0',
 'GPX3': 'Rejeced H0',
 'IPO9': 'Rejeced H0',
 'SHKBP1': 'Rejeced H0',
 'ARHGAP40': 'Rejeced H0',
 'ENO3': 'Rejeced H0',
 'COX19': 'Rejeced H0',
 'LMF1': 'Rejeced H0',
 'S100P': 'Rejeced H0',
 'RDH14': 'Rejec

In [158]:
len(rejected_genes_paired)

17653

In [141]:
p=[]
gene_name=[]
for gene, pvalue in p_value_rel_dict.items():
    p.append(pvalue)
    gene_name.append(gene)


In [142]:
p

[2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,
 2.1336659955541074e-07,


In [143]:
len(p)

17653

In [144]:
q_fdr = multi.multipletests(p, method = 'fdr_bh')[1]

In [145]:
q_fdr

array([2.133666e-07, 2.133666e-07, 2.133666e-07, ..., 2.133666e-07,
       2.133666e-07, 2.133666e-07])

In [146]:
len(q_fdr)

17653

In [150]:
gene_names=[gene_name[x] for x in range(len(p)) if q_fdr[x] <= 0.05]

In [151]:
gene_names

['HIST3H2A',
 'LIN7B',
 'LXN',
 'CNKSR2',
 'SCML1',
 'AC024592.12',
 'GSDMD',
 'AKR1C1',
 'C3orf62',
 'CRISPLD2',
 'DOCK5',
 'SLC33A1',
 'GLI1',
 'STK17B',
 'RP5-966M1.6',
 'VPS52',
 'HAGHL',
 'CDC37L1',
 'KRT10',
 'AEN',
 'CYHR1',
 'VWA1',
 'EXOSC9',
 'TNFAIP8L2',
 'PFDN2',
 'TCP1',
 'ALG9',
 'RPS15A',
 'PMEL',
 'SFMBT1',
 'CYBA',
 'GINS1',
 'GPX3',
 'IPO9',
 'SHKBP1',
 'ARHGAP40',
 'ENO3',
 'COX19',
 'LMF1',
 'S100P',
 'RDH14',
 'GDE1',
 'LSM14B',
 'TLE2',
 'ACOT8',
 'EIF4EBP3',
 'LRP2BP',
 'PELO',
 'CDIPT',
 'C19orf44',
 'PABPN1',
 'NFKBIB',
 'ZWINT',
 'DYNLL1',
 'NRXN3',
 'FYTTD1',
 'CAPRIN1',
 'AOC3',
 'KIFC3',
 'DOCK11',
 'NENF',
 'TMEM231',
 'DNAH17',
 'DCHS1',
 'ZFYVE19',
 'NTRK1',
 'CCDC62',
 'HS3ST4',
 'SLC5A8',
 'WDR53',
 'NLN',
 'TBC1D31',
 'NOXRED1',
 'ZNF597',
 'RNF114',
 'JOSD2',
 'SFRP2',
 'PNMA2',
 'CDH10',
 'C1orf158',
 'HENMT1',
 'STX16',
 'PTX3',
 'BFAR',
 'MEP1B',
 'SHROOM3',
 'NUP107',
 'HS6ST1',
 'BRWD3',
 'FGF11',
 'CCDC138',
 'FAM193B',
 'BATF',
 'COL4A2',
 'AG

In [152]:
len(gene_names)

17653